In [1]:
import itertools
import joblib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import pipeline

import poker_coach

In [2]:
hero_range = range(0, 101, 1)
villain_range = range(0, 101, 1)
n_times = 100000

In [3]:
%%time

# train = pd.read_csv(r"poker_coach\data\linear_model_train.csv", index_col=0)
# train.head()

prod = itertools.product(hero_range, villain_range)

train = pd.DataFrame()

for h, v in prod:
    
    # Get hand from ranking.
    hero_hand = poker_coach.equity.percentage_descr(h)

    # Calculate equity agains villain range.
    equity = poker_coach.Scenario.eval_ranges(hero_hand, [v], times=n_times)
    
    train = train.append({"Hero": h, "Villain": v, "Equity": equity[0]}, ignore_index=True)
    
train.to_csv(r"poker_coach\data\linear_model_train.csv")
train.head()

UsageError: Line magic function `%%time` not found.


In [ ]:
%%time

# test = pd.read_csv(r"poker_coach\data\linear_model_test.csv", index_col=0)
# test.head()

test_size = 0.25

test = pd.DataFrame()
for _ in range(int(train.shape[0] * test_size)):
    
    # Choose randomly hero and villain ranges.
    h = np.random.randint(min(hero_range), max(hero_range))
    hero_hand = poker_coach.equity.percentage_descr(h)
    v = np.random.randint(min(villain_range), max(villain_range))
    
    # Calculate equity agains villain range.
    equity = poker_coach.Scenario.eval_ranges(hero_hand, [v], times=n_times)
    
    test = test.append({"Hero": h, "Villain": v, "Equity": equity[0]}, ignore_index=True)
    
test.to_csv(r"poker_coach\data\linear_model_test.csv")
test.head()

In [ ]:
x_train = train[["Hero", "Villain"]]
y_train = train[["Equity"]]

x_test = test[["Hero", "Villain"]]
y_test = test[["Equity"]]

pipe = pipeline.Pipeline(
    [
        ("random forest", ensemble.RandomForestRegressor(100)),
    ]
)

pipe.fit(x_train, y_train.values.ravel())
pipe.score(x_test, y_test)

In [ ]:
joblib.dump(pipe, 'model.pkl') 

In [ ]:
test["Predicted Equity"] = pipe.predict(x_test)